In [23]:
import pandas as pd
import math
import os
import numpy as np
import matplotlib as plt
import datetime

#library for representing the graphs
import networkx as nx

#library for Bellman Ford Algorithm
import bellmanford as bf

#### Verifying Price Data
<font color=#333333>I define a path where are stored all the datasets.</font>
<font color=#333333>The command _os.listdir_ makes a list of all the files found in the directory</font>

In [24]:
PATH_TO_FILES = "/home/pfl/Ironhack/project_Searching_for_Arbitrage_Opportunities_in_the_FX_Market_on_the_paper/datasets_csv"
filescsv = os.listdir(PATH_TO_FILES)

In [25]:
filescsv

['Xasmus.17302.PERIOD_M1.EURNZDmicro.csv',
 'Xasmus.23925.PERIOD_M1.USDMXNmicro.csv',
 'Xasmus.25659.PERIOD_M1.AUDCADmicro.csv',
 'Xasmus.27030.PERIOD_M1.CADJPYmicro.csv',
 'Xasmus.1343.PERIOD_M1.AUDCHFmicro.csv',
 'Xasmus.9677.PERIOD_M1.USDSGDmicro.csv',
 'Xasmus.2711.PERIOD_M1.EURGBPmicro.csv',
 'Xasmus.13341.PERIOD_M1.GBPCADmicro.csv',
 'Xasmus.27552.PERIOD_M1.NZDCADmicro.csv',
 'Xasmus.3830.PERIOD_M1.GBPNOKmicro.csv',
 'Xasmus.2386.PERIOD_M1.USDCADmicro.csv',
 'Xasmus.2367.PERIOD_M1.EURNOKmicro.csv',
 'Xasmus.2004.PERIOD_M1.GBPSGDmicro.csv',
 'Xasmus.18640.PERIOD_M1.GBPDKKmicro.csv',
 'Xasmus.15607.PERIOD_M1.GBPNZDmicro.csv',
 'Xasmus.2091.PERIOD_M1.EURSGDmicro.csv',
 'Xasmus.29230.PERIOD_M1.GBPCHFmicro.csv',
 'Xasmus.6999.PERIOD_M1.SGDJPYmicro.csv',
 'Xasmus.10086.PERIOD_M1.CADCHFmicro.csv',
 'Xasmus.13730.PERIOD_M1.EURAUDmicro.csv',
 'Xasmus.11037.PERIOD_M1.AUDJPYmicro.csv',
 'Xasmus.2309.PERIOD_M1.NZDJPYmicro.csv',
 'Xasmus.2834.PERIOD_M1.USDHUFmicro.csv',
 'Xasmus.18723.PERIOD_

In [3]:
CURRENT_PATH = os.getcwd()
os.chdir(PATH_TO_FILES)
os.getcwd()

'/home/pfl/Ironhack/project_Searching_for_Arbitrage_Opportunities_in_the_FX_Market_on_the_paper/datasets_csv'

Arrived to the conclusion that the times in the rows are not the same for each pair of currency, there is a part of data cleaning and wrangling in order to be coherent 

In [21]:
n_nans = []
count = []
first_datetime = []

for file in range(4): #len(filescsv)
    df = pd.read_csv(filescsv[file])
    n_nans.append(df["Price"].isna().sum())
    count.append(df["time"].count())
    first_datetime.append(df["time"][0])
    print(f"{file}  {filescsv[file]}","\t"+"Number of NaNs:",n_nans[-1],"\t",count[-1],"\t",first_datetime[-1])

0  Xasmus.17302.PERIOD_M1.EURNZDmicro.csv 	Number of NaNs: 0 	 1394991 	 2015.01.02 08:06:00
1  Xasmus.23925.PERIOD_M1.USDMXNmicro.csv 	Number of NaNs: 0 	 1374169 	 2015.01.02 08:07:00
2  Xasmus.25659.PERIOD_M1.AUDCADmicro.csv 	Number of NaNs: 0 	 1394461 	 2015.01.02 08:06:00
3  Xasmus.27030.PERIOD_M1.CADJPYmicro.csv 	Number of NaNs: 0 	 1394806 	 2015.01.02 08:06:00


<font color=666666>Text</font>

<font color=#4c4c4c>Text</font>

In [14]:
# to try with less values write <filecsv[:#number]> or <filecsv> to write all the values
files_info_df = pd.DataFrame({"file_name":filescsv[:4],"number_nans":n_nans,"number_rows":count,"first_datetime":first_datetime})
files_info_df.head()

,file_name,number_nans,number_rows,first_datetime
0,Xasmus.17302.PERIOD_M1.EURNZDmicro.csv,0,1394991,2015.01.02 08:06:00
1,Xasmus.23925.PERIOD_M1.USDMXNmicro.csv,0,1374169,2015.01.02 08:07:00
2,Xasmus.25659.PERIOD_M1.AUDCADmicro.csv,0,1394461,2015.01.02 08:06:00
3,Xasmus.27030.PERIOD_M1.CADJPYmicro.csv,0,1394806,2015.01.02 08:06:00


In [16]:
reference_rows = files_info_df[files_info_df["number_rows"] == files_info_df["number_rows"].min()]

In [30]:
point_reference = reference_rows["file_name"].index[0]

In [31]:
point_reference

1

### Creating csv file including all the pairs forex

In [ ]:
df_raw = pd.read_csv(point_reference) #Here the number inside "filescsv[???] is the first number appeared before."
currency = {"time": df_raw["time"], df_raw["Symbol()"][0][:-5]: df_raw["Price"]}
currency_df = pd.DataFrame(currency)
currency_df.head()

In [ ]:
new_filescsv = filescsv[:point_reference] + filescsv[(point_reference+1):] #here the same, the numbers can vary

In [ ]:
for i in range(len(new_filescsv)):
    print(f"{i}: {new_filescsv[i]}")
    df_raw = pd.read_csv(new_filescsv[i])
    df_raw.set_index("time", drop=True, inplace=True)
    df_raw = df_raw.reindex(currency_df["time"])
    name = df_raw["Symbol()"].dropna()
    df_raw.reset_index(inplace=True)
    currency_df[name[name.index[0]][:-5]] = df_raw["Price"]

In [ ]:
currency_df.head()

In [ ]:
currency_df.to_csv("arbitrage_forex_48_pairs_currency_v2.csv")

### Reading currency_df

In [2]:
PATH_TO_FILES = "/home/pfl/Ironhack/project_Searching_for_Arbitrage_Opportunities_in_the_FX_Market_on_the_paper/results_csv"

In [3]:
CURRENT_PATH = os.getcwd()
os.chdir(PATH_TO_FILES)
os.getcwd()

'/home/pfl/Ironhack/project_Searching_for_Arbitrage_Opportunities_in_the_FX_Market_on_the_paper/results_csv'

In [5]:
currency_df = pd.read_csv("arbitrage_forex_48_pairs_currency_v2.csv", index_col=0)

In [6]:
currency_df.shape[0]

1365805

In [13]:
small_df = currency_df.iloc[0:365805]

### Connection with the database (MySQL)

In [7]:
import pymysql
from sqlalchemy import create_engine

driver = 'mysql+pymysql:'
user = 'root'
password = '12345678'
ip = '127.0.0.1'
database = 'project_Arbitrage_Opportunities_FX_Market'

connection_string = f'{driver}//{user}:{password}@{ip}/{database}'
engine = create_engine(connection_string)

In [3]:
import time, sys
from IPython.display import clear_output

def update_progress(progress):
    bar_length = 20
    if isinstance(progress, int):
        progress = float(progress)
    if not isinstance(progress, float):
        progress = 0
    if progress < 0:
        progress = 0
    if progress >= 1:
        progress = 1

    block = int(round(bar_length * progress))

    clear_output(wait = True)
    text = "Progress: [{0}] {1:.1f}%".format( "#" * block + "-" * (bar_length - block), progress * 100)
    print(text)

In [6]:
number_of_elements = 1000

for i in range(number_of_elements):
    x = i*2
    print(x)
    #time.sleep(0.01) #Replace this with a real computation
    update_progress(i / number_of_elements)
print(x)
update_progress(1)

Progress: [####################] 100.0%


In [19]:
a = 5
count = 0
for split in np.split(currency_df,a):
    #print(count)
    #print(split)
    split.to_sql(f"partial_{count}_pairs_of_currencies",engine,if_exists = 'replace')
    count+=1
    time.sleep(15)
    #update_progress(i / number_of_elements)
#update_progress(1)

In [8]:
currency_df.to_sql("full_pairs_of_currencies",engine,chunksize=150000)

In [10]:
query = """
            SELECT table_schema "database", sum(data_length + index_length)/1024/1024 "size in MB" FROM information_schema.TABLES WHERE table_schema='project_Arbitrage_Opportunities_FX_Market';
"""

In [13]:
space_used_database_pair_of_currencies = pd.read_sql(query,engine)

In [14]:
space_used_database_pair_of_currencies

,database,size in MB
0,project_Arbitrage_Opportunities_FX_Market,564.5625


### Creating the Pairs Table (18 currencies)

In [ ]:
list_headers = currency_df.columns.tolist()
list_headers = list_headers[1:]
list_currencies = []
for position in list_headers:
    list_currencies.append(position[:3])
    list_currencies.append(position[3:])
set_currencies = set(list_currencies)
unique_currencies = sorted(list(set_currencies))

In [ ]:
pairs_table_df = pd.DataFrame(0, index=unique_currencies, columns=unique_currencies)

In [ ]:
d = {currency: [0] for currency in unique_currencies}
pairs_table_df = pd.DataFrame(data=d, index=unique_currencies)
pairs_table_df = pairs_table_df.sort_index()
pairs_table_df.columns = pairs_table_df.index

In [ ]:
def deleting_rows_columns_only_nans(df):
    columns_to_remove = []
    for column in df.columns:
        dropping = df[column].drop(column)
        if dropping.isna().all():
            columns_to_remove.append(column)
    df.drop(columns_to_remove, axis=1, inplace=True)
    df.drop(columns_to_remove, inplace=True)
    return df

In [ ]:
SAVING_PATH = "/home/pau/Ironhack/week9/final_project/Project-Week-8-Final-Project/your-project/code/possibles_arbitrages_currencies"

for index in range(len(currency_df["time"][0:])): #I wrote "20" in this case because I wanted to iterate only the first 20 tables, to iterate until the end I have to write "[0:]"
    for row in pairs_table_df.columns:
        for column in pairs_table_df.index:
            try:
                if row == column:
                    pairs_table_df.loc[row,column] = 1
                else:
                    pairs_table_df.loc[row,column] = currency_df.loc[index][1:][row+column]
                    pairs_table_df.loc[column,row] = 1/currency_df.loc[index][1:][row+column]    # perquè és "df.iloc" i no "df.loc" ????
            except:
                if pairs_table_df.loc[row,column] == 0:
                    pairs_table_df.loc[row,column] = np.NaN
    print(f"{index}   Iteration {currency_df.time[index]}")
    pairs_table_df = deleting_rows_columns_only_nans(pairs_table_df)
    pairs_table_df.to_csv(f"{SAVING_PATH}/{currency_df.time[index]}_possible_arbitrage.csv")
    pairs_table_df = pd.DataFrame(0, index=unique_currencies, columns=unique_currencies)

### Arbitrage opportunities

In [ ]:
PATH_TO_FILES = "/home/pau/Ironhack/week9/final_project/Project-Week-8-Final-Project/your-project/code/possibles_arbitrages_currencies"
filescsv = os.listdir(PATH_TO_FILES)

In [ ]:
CURRENT_PATH = os.getcwd()
os.chdir(PATH_TO_FILES)
os.getcwd()

### Bellman Ford Algorithm to find the arbitrage opportunity

In [ ]:
def create_graph(currencies):
    G = nx.DiGraph()
    for series in currencies:
        G.add_nodes_from(series.index)
        # """series.index[i] = ROWS""" """series.name = COLUMNS"""
        G.add_edges_from([(series.index[i], series.name, {'weight':-np.log(series[i])}) for i in range(len(series))])
    return G

In [ ]:
def columns_formatter(df):
    series = []
    for i in df.columns:
        values = df[i].apply(lambda x: x if x !=1.0 else np.nan)
        values = values.dropna()
        series.append(values)
    return series 

In [ ]:
def calculation_profit(clist):
    total = 1
    for i in range(1,len(clist)):
        total *= arb_df.loc[clist[i-1]][clist[i]]
    return total

In [ ]:
def print_profit_results(results,unique_paths):
    currencies_paths = results[1]
    profit = calculation_profit(currencies_paths)
    if (profit > 1.00005) and (currencies_paths not in unique_paths):
        print(f"Path: {currencies_paths}")
        for i in range(1,len(currencies_paths)):
            print(f"{currencies_paths[i-1]}-{currencies_paths[i]}",arb_df.loc[currencies_paths[i-1]][currencies_paths[i]]) #arb_df.loc[ROWS][COLUMNS]
        print(f"Profit: {profit-1}")
        print("\n")
        unique_paths.append(currencies_paths)
    return unique_paths

In [ ]:
timestamp = []
path_id = []
complete_path = []
source = []
target = []
change = []
profit_lst = []

path_counter = 0
for file in range(len(filescsv)):
#for file in range(0,8):
    print(f"{file}   ############################## {filescsv[file]} ##############################")
    arb_df = pd.read_csv(filescsv[file], index_col=0)
    currencies = columns_formatter(arb_df)
    G = create_graph(currencies)
    unique_paths = []
    for i in arb_df.index:
        results = bf.bellman_ford(G, source=i, target=i, weight="weight")
        currencies_paths = results[1]
        profit = calculation_profit(currencies_paths)
        if (profit > 1.00005) and (currencies_paths not in unique_paths):
            #print(f"Path: {currencies_paths}")
            for j in range(1,len(currencies_paths)):
                timestamp.append(filescsv[file])
                source.append(currencies_paths[j-1])
                target.append(currencies_paths[j])
                change.append(arb_df.loc[currencies_paths[j-1]][currencies_paths[j]])
                #print(f"{source[-1]}-{target[-1]}",change[-1]) #arb_df.loc[ROWS][COLUMNS]
                path_id.append(path_counter)
                complete_path.append(currencies_paths)
                profit_lst.append(profit-1)
            path_counter += 1
            #print(f"Profit: {profit-1}")
            #print("\n")
            unique_paths.append(currencies_paths)

In [ ]:
x = pd.DataFrame({"timestamp":timestamp,"path_id":path_id,"complete_path":complete_path,"source":source,"target":target,"change":change,"profit":profit_lst})
print(len(timestamp))
print(len(path_id))
print(len(complete_path))
print(len(source))
print(len(target))
print(len(change))
print(len(profit_lst))

In [ ]:
x